control_v11p_sd15_canny [d14c016b]

In [51]:
import json
import requests
import io
import base64
from PIL import Image, PngImagePlugin

url = "http://127.0.0.1:7860"

#open image file using PIL and then covert it to base64 string
# Open the image
# base64_string = ""
# with open('dog.jpg', 'rb') as image_file:
#     base64_bytes = base64.b64encode(image_file.read())
#     print(base64_bytes)

#     base64_string = base64_bytes.decode()

from io import BytesIO

image_path = 'top_wear.png'
image = Image.open(image_path)
buffered = BytesIO()
image.save(buffered, format="PNG")
img_str = base64.b64encode(buffered.getvalue()).decode()



payload = {
    "prompt": "full-body potrait of female model Casual Puff Sleeves Floral Print Women Pink Top white background studio lighting 2k resolution",
	"negative_prompt": "(deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), text, close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck" ,
    "steps": 20 ,
	"alwayson_scripts": {
    "controlnet": {
      "args": [
        {
		  "lowvram": "True",
          "input_image": img_str, 
		  "module" : "reference_adain+attn",
		  "pixel_perfect": "True",
        }
      ]
    }
  }
}

response = requests.post(url=f'{url}/sdapi/v1/txt2img', json=payload)

r = response.json()

k = 0
for i in r['images']:
	image = Image.open(io.BytesIO(base64.b64decode(i.split(",",1)[0])))

    # png_payload = {
    #     "image": "data:image/png;base64," + i
    # }
    # response2 = requests.post(url=f'{url}/sdapi/v1/png-info', json=png_payload)

    # pnginfo = PngImagePlugin.PngInfo()
    # pnginfo.add_text("parameters", response2.json().get("info"))
    # image.save('output.png', pnginfo=pnginfo)
	k += 1
	image.save(f'output{k}.png')

In [40]:


from io import BytesIO

image_path = 'dog.jpg'
image = Image.open(image_path)
buffered = BytesIO()
image.save(buffered, format="PNG")
img_str = base64.b64encode(buffered.getvalue()).decode()




print(img_str)

...
